In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
userId = 'musiclove'
url = "https://www.acmicpc.net/user/{}".format(userId)
response = requests.get(url, headers=headers)
if response.status_code != 200:
    print(response.status_code)

html = response.text
soup = BeautifulSoup(html, 'html.parser')
snum = soup.select_one('#u-result-4')
print(snum)

NameError: name 'userId' is not defined

In [36]:
userId = 'annekim1216'
url = "https://www.acmicpc.net/status?user_id={userId}&result_id=4".format(userId=userId)
sbDict = []
cnt = 1
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
while True:
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.select_one('tbody')
        tr = table.select('tr')
        for i in range(len(tr)):
            td = tr[i].select('td')
            sbNum = td[0].text
            uId = td[1].text
            pId = td[2].text
            result = td[3].text
            memory = td[4].text
            time = td[5].text
            lang = td[6].text
            codeLen = td[7].text
            sbTime = td[8].select_one('a')['data-timestamp']
            sbDict.append({'sbNum':sbNum, 'uId':uId, 'pId':pId, 'result':result, 'memory':memory, 'time':time, 'lang':lang, 'codeLen':codeLen, 'sbTime':sbTime})
        try:
            nextPage = 'https://www.acmicpc.net' + soup.select_one('#next_page')['href']
            print(nextPage)
        except Exception:
            break
        response = requests.get(nextPage, headers=headers)
        print(cnt)
        cnt = cnt + 1
    else: print(response.status_code)
print(sbDict)

https://www.acmicpc.net/status?user_id=annekim1216&result_id=4&top=29349291
1
https://www.acmicpc.net/status?user_id=annekim1216&result_id=4&top=22686374
2
https://www.acmicpc.net/status?user_id=annekim1216&result_id=4&top=16086908
3
https://www.acmicpc.net/status?user_id=annekim1216&result_id=4&top=15501042
4
https://www.acmicpc.net/status?user_id=annekim1216&result_id=4&top=14715986
5
https://www.acmicpc.net/status?user_id=annekim1216&result_id=4&top=13994459
6
https://www.acmicpc.net/status?user_id=annekim1216&result_id=4&top=12593747
7
https://www.acmicpc.net/status?user_id=annekim1216&result_id=4&top=12229343
8
[{'sbNum': '39373387', 'uId': 'annekim1216', 'pId': '1922', 'result': '맞았습니다!!', 'memory': '4460', 'time': '32', 'lang': 'C++17', 'codeLen': '900', 'sbTime': '1645380383'}, {'sbNum': '39373109', 'uId': 'annekim1216', 'pId': '2252', 'result': '맞았습니다!!', 'memory': '6008', 'time': '32', 'lang': 'C++17', 'codeLen': '756', 'sbTime': '1645379183'}, {'sbNum': '39372186', 'uId': 'a

In [37]:
len(sbDict)

165

In [20]:
import pandas as pd
import json
import requests

In [5]:
def toJson(dir, dict):
    with open(dir, 'w', encoding='utf-8') as file:
        json.dump(dict, file, ensure_ascii=False, indent='\t')

def loadJson(dir):
    with open(dir, 'r', encoding='utf-8') as file:
        return json.load(file)

In [9]:
dir = 'D:/AlgorithmProblemRecommender/Recommenders/recommend/'
userId = "musiclove529"
categoryDF = pd.read_csv(dir + 'category.csv', index_col=0, dtype=str, encoding='utf-8')

file = dir + 'data/{userId}/{userId}.json'.format(userId=userId)
userDF = pd.read_json(file, dtype = str, encoding='utf-8')
# 맞은 문제만 추출
userIndex = list(userDF[userDF['result']=='맞았습니다!!'].index)
userIndex = userIndex + list(userDF[userDF['result']=='100점'].index)
userDF = userDF.loc[userIndex]
# 문제셋에 해당하는 문제만 추출
userDF = userDF[userDF['pId'].isin(categoryDF['pId'])]

# erase duplications
userDF.drop_duplicates(subset=['pId'], keep='last', inplace=True, ignore_index=True)
userDF

,sbNum,uId,pId,result,memory,time,lang,codeLen,sbTime
0,44123685,musiclove529,18111,맞았습니다!!,37112,704,Java 11,1654,1654248077
1,44018577,musiclove529,9012,맞았습니다!!,14384,136,Java 11,859,1654006520
2,43793489,musiclove529,4949,맞았습니다!!,20588,352,Java 11,1086,1653503831
3,43793125,musiclove529,10026,맞았습니다!!,17956,184,Java 11,2608,1653501526
4,43779742,musiclove529,2164,맞았습니다!!,45616,188,Java 11,504,1653478075
...,...,...,...,...,...,...,...,...,...
167,15502482,musiclove529,1330,맞았습니다!!,14244,104,Java 8,324,1570199763
168,15502384,musiclove529,2588,맞았습니다!!,14240,116,Java 8,368,1570199439
169,15502158,musiclove529,1008,맞았습니다!!,14500,108,Java 8,233,1570198629
170,37338320,musiclove529,15829,100점,14424,132,Java 11,569,1641801465


In [31]:
recentSlovedProblems = list(userDF.head(100)['pId'])
recentSlovedProblems

['18111',
 '9012',
 '4949',
 '10026',
 '2164',
 '1874',
 '1253',
 '11660',
 '1010',
 '1697',
 '2023',
 '1644',
 '11659',
 '1806',
 '1717',
 '2252',
 '2609',
 '1966',
 '10866',
 '1436',
 '2606',
 '1654',
 '9095',
 '1003',
 '1149',
 '1463',
 '2512',
 '10815',
 '2805',
 '7576',
 '2693',
 '11399',
 '2012',
 '1260',
 '18352',
 '1018',
 '10845',
 '10828',
 '11050',
 '1259',
 '1475',
 '2960',
 '14500',
 '1931',
 '2798',
 '1978',
 '2775',
 '2292',
 '2231',
 '1085',
 '2908',
 '1157',
 '1152',
 '1316',
 '4673',
 '2675',
 '10870',
 '10872',
 '2676',
 '2563',
 '10709',
 '7326',
 '1476',
 '2108',
 '1212',
 '2751',
 '10539',
 '11772',
 '15803',
 '11651',
 '10825',
 '1181',
 '12354',
 '10809',
 '11441',
 '11109',
 '2822',
 '14655',
 '1924',
 '11131',
 '10808',
 '15725',
 '15780',
 '1427',
 '2490',
 '11367',
 '2869',
 '3047',
 '3034',
 '15719',
 '11719',
 '11718',
 '1188',
 '10989',
 '10814',
 '2839',
 '17520',
 '15552',
 '1920',
 '2506']

In [32]:
problemStr = ""
for p in recentSlovedProblems:
    problemStr += p + ","
problemStr = problemStr[:-1]
problemStr

'18111,9012,4949,10026,2164,1874,1253,11660,1010,1697,2023,1644,11659,1806,1717,2252,2609,1966,10866,1436,2606,1654,9095,1003,1149,1463,2512,10815,2805,7576,2693,11399,2012,1260,18352,1018,10845,10828,11050,1259,1475,2960,14500,1931,2798,1978,2775,2292,2231,1085,2908,1157,1152,1316,4673,2675,10870,10872,2676,2563,10709,7326,1476,2108,1212,2751,10539,11772,15803,11651,10825,1181,12354,10809,11441,11109,2822,14655,1924,11131,10808,15725,15780,1427,2490,11367,2869,3047,3034,15719,11719,11718,1188,10989,10814,2839,17520,15552,1920,2506'

In [33]:
apiUrl = 'https://solved.ac/api/v3/problem/lookup?problemIds='
response = requests.get(apiUrl+problemStr).json()
response

[{'problemId': 1003,
  'titleKo': '피보나치 함수',
  'titles': [{'language': 'ko',
    'languageDisplayName': 'ko',
    'title': '피보나치 함수',
    'isOriginal': True}],
  'isSolvable': True,
  'isPartial': False,
  'acceptedUserCount': 35997,
  'level': 8,
  'votedUserCount': 112,
  'sprout': False,
  'givesNoRating': False,
  'isLevelLocked': False,
  'averageTries': 3.1741,
  'official': True,
  'tags': [{'key': 'dp',
    'isMeta': False,
    'bojTagId': 25,
    'problemCount': 2498,
    'displayNames': [{'language': 'ko',
      'name': '다이나믹 프로그래밍',
      'short': '다이나믹 프로그래밍'},
     {'language': 'en', 'name': 'dynamic programming', 'short': 'dp'}]}]},
 {'problemId': 1010,
  'titleKo': '다리 놓기',
  'titles': [{'language': 'ko',
    'languageDisplayName': 'ko',
    'title': '다리 놓기',
    'isOriginal': True}],
  'isSolvable': True,
  'isPartial': False,
  'acceptedUserCount': 19978,
  'level': 6,
  'votedUserCount': 83,
  'sprout': False,
  'givesNoRating': False,
  'isLevelLocked': False,
  'ave

In [34]:
tagCnt = {}
for p in response:
    tags = p['tags']
    for t in tags:
        tagName = t['displayNames'][0]['name']
        keys = tagCnt.keys()
        if tagName not in keys:
            tagCnt[tagName] = 1
        else:
            tagCnt[tagName] += 1
tagCnt = dict(sorted(tagCnt.items(), key=lambda x: x[1], reverse=True))
tagCnt

{'수학': 34,
 '구현': 34,
 '문자열': 16,
 '정렬': 15,
 '자료 구조': 12,
 '브루트포스 알고리즘': 9,
 '다이나믹 프로그래밍': 7,
 '정수론': 7,
 '그래프 이론': 7,
 '사칙연산': 7,
 '이분 탐색': 6,
 '너비 우선 탐색': 6,
 '그래프 탐색': 6,
 '그리디 알고리즘': 5,
 '소수 판정': 4,
 '스택': 4,
 '기하학': 3,
 '두 포인터': 3,
 '깊이 우선 탐색': 3,
 '에라토스테네스의 체': 3,
 '매개 변수 탐색': 3,
 '큐': 3,
 '누적 합': 3,
 '조합론': 2,
 '시뮬레이션': 2,
 '많은 조건 분기': 2,
 '해시를 사용한 집합과 맵': 1,
 '분리 집합': 1,
 '백트래킹': 1,
 '위상 정렬': 1,
 '유클리드 호제법': 1,
 '피타고라스 정리': 1,
 '덱': 1,
 '미적분학': 1,
 '파싱': 1,
 '데이크스트라': 1}

In [35]:
topTags = list(tagCnt.keys())[:10]
topTags

['수학',
 '구현',
 '문자열',
 '정렬',
 '자료 구조',
 '브루트포스 알고리즘',
 '다이나믹 프로그래밍',
 '정수론',
 '그래프 이론',
 '사칙연산']

In [39]:
reportData = []
for t in topTags:
    dic = {'tag':t, 'Bronze':0, 'Silver':0, "Gold":0, 'Platinum':0}
    for p in response:
        level = p['level']
        tags = p['tags']
        for tag in tags:
            tagName = tag['displayNames'][0]['name']
            if tagName == t:
                if level<6: dic['Bronze']+=1
                elif level<11: dic['Silver']+=1
                elif level<16: dic['Gold']+=1
                elif level<21: dic['Platinum']+=1
    reportData.append(dic)
reportData


[{'tag': '수학', 'Bronze': 21, 'Silver': 10, 'Gold': 3, 'Platinum': 0},
 {'tag': '구현', 'Bronze': 25, 'Silver': 8, 'Gold': 1, 'Platinum': 0},
 {'tag': '문자열', 'Bronze': 10, 'Silver': 6, 'Gold': 0, 'Platinum': 0},
 {'tag': '정렬', 'Bronze': 0, 'Silver': 14, 'Gold': 1, 'Platinum': 0},
 {'tag': '자료 구조', 'Bronze': 0, 'Silver': 10, 'Gold': 2, 'Platinum': 0},
 {'tag': '브루트포스 알고리즘', 'Bronze': 2, 'Silver': 6, 'Gold': 1, 'Platinum': 0},
 {'tag': '다이나믹 프로그래밍', 'Bronze': 1, 'Silver': 6, 'Gold': 0, 'Platinum': 0},
 {'tag': '정수론', 'Bronze': 0, 'Silver': 4, 'Gold': 3, 'Platinum': 0},
 {'tag': '그래프 이론', 'Bronze': 0, 'Silver': 4, 'Gold': 3, 'Platinum': 0},
 {'tag': '사칙연산', 'Bronze': 7, 'Silver': 0, 'Gold': 0, 'Platinum': 0}]